This file is for pure exploration applied to other projects and repositories. I am making use of the imported ecoinvent version to explore.

In [ ]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

In [ ]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

In [ ]:
bd.databases

In [ ]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
db_template = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

In [ ]:
ei_clca.search('steel')

In [ ]:
steel = ei_clca.get(name = 'steel production, electric, chromium steel 18/8', location = "RER", unit = 'kilogram')
cast_iron = ei_clca.get(name = 'cast iron production', location = "RER", unit = 'kilogram')

welding_arc = ei_clca.get(name = 'welding, arc, steel', location = "RER", unit = 'meter')
welding_gas = ei_clca.get(name = 'welding, gas, steel', location = "RER", unit = 'meter')

In [ ]:
# export the LCI to excel in an overview format
bi.export.excel.write_lci_excel(ei_clca.name,
                                objs=[welding_arc, welding_gas],
                                dirpath=Path.cwd())

In [ ]:
# quickly print all exchanges of the process defined above, sorted by amount (large to small)
exchanges = sorted(welding_arc.exchanges(), key=lambda exc: exc['amount'], reverse=True)
for exc in exchanges:
    print(exc)

In [ ]:
# printing all methods that contain a specific keyword, e.g. "climate change"
climate_methods = [m for m in bd.methods if any("gwp100" in str(part).lower() for part in m)]
for method in climate_methods:
    print(method)

In [ ]:
# selecting the impact assessment methods that we want to use in our analysis
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')

### 1. Level: comparing the final results for steel and cast iron

In [ ]:
# One option to do a contribution analysis is to use the recursive calculation method
bwa.print_recursive_calculation(welding_arc,
lcia_method=lcia_gwp100,max_level=3,cutoff=0.05)

In [ ]:
# Quick LCIA calculation
arcwelding_lca = welding_arc.lca(lcia_gwp100)
arcwelding_lca.score

In [ ]:
gaswelding_lca = welding_gas.lca(lcia_gwp100)
gaswelding_lca.score

In [ ]:
# Another option is to use the recursive calculation to an object, which returns a DataFrame
arcwelding_lca_ca = bwa.utils.recursive_calculation_to_object(welding_arc,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.01,
                                          as_dataframe=True,
                                          )

In [ ]:
# The goal here is to conduct the same analysis for the ecoinvent process so we can compare the contributions later
gaswelding_lca_ca = bwa.utils.recursive_calculation_to_object(welding_gas,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.01,
                                          as_dataframe=True,
                                          )

In [ ]:
# Filter out the parent processes that are not relevant for the contribution analysis
arcwelding_lca_ca = arcwelding_lca_ca.dropna(subset='parent') 
gaswelding_lca_ca = gaswelding_lca_ca.dropna(subset='parent')



In [ ]:
combined_df = pd.concat([arcwelding_lca_ca.set_index('name').score,gaswelding_lca_ca.set_index('name').score],
                        keys=['arc welding','gas welding'],
                        names=['activity','contributor'])
combined_df

In [ ]:
combined_df = combined_df.groupby(level=combined_df.index.names).sum()

ax = combined_df.unstack().plot.barh(
    stacked=True,
    title='Main contributors to global warming',
    xlabel='kg CO2eq',
    figsize=(10,6)
)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#

### 2. Level: looking at the pig iron

In [ ]:
ei_clca.search('welding')

In [ ]:
pig_iron = ei_clca.get(name = 'pig iron production', location = "RER", unit = 'kilogram')

In [ ]:
ferronickel = ei_clca.get(name = 'ferronickel production', location = "GLO", unit = 'kilogram')

In [ ]:
# export the LCI to excel in an overview format
bi.export.excel.write_lci_excel(ei_clca.name,
                                objs=[pig_iron],
                                dirpath=Path.cwd())

In [ ]:
pig_iron_lca = pig_iron.lca(lcia_gwp100)
pig_iron_lca.score

In [ ]:
# deeper contribution analysis for steel vs cast iron
bwa.print_recursive_calculation(steel,
lcia_method=lcia_gwp100,max_level=4,cutoff=0.03)

In [ ]:
# deeper contribution analysis for steel vs cast iron
bwa.print_recursive_calculation(ferronickel,
lcia_method=lcia_gwp100,max_level=2,cutoff=0.03)

In [ ]:
# deeper contribution analysis for steel vs cast iron
bwa.print_recursive_calculation(cast_iron,
lcia_method=lcia_gwp100,max_level=4,cutoff=0.03)

In [ ]:
# deeper contribution analysis for steel vs cast iron
bwa.print_recursive_calculation(pig_iron,
lcia_method=lcia_gwp100,max_level=3,cutoff=0.01)